In [1]:
import findspark
findspark.init()

In [2]:
from pyspark.sql import SparkSession

In [ ]:
spark = SparkSession.builder.appName("lrex").getOrCreate()


In [ ]:
from pyspark.ml.regression import LinearRegression

In [5]:
training = spark.read.format('libsvm').load("linearregressiondata/sample_linear_regression_data.txt")

In [6]:
training.describe().show()

+-------+-------------------+
|summary|              label|
+-------+-------------------+
|  count|                501|
|   mean|0.25688882219498976|
| stddev| 10.317884030544564|
|    min|-28.571478869743427|
|    max|  27.78383192005107|
+-------+-------------------+



In [7]:
training.show()

+-------------------+--------------------+
|              label|            features|
+-------------------+--------------------+
| -9.490009878824548|(10,[0,1,2,3,4,5,...|
| 0.2577820163584905|(10,[0,1,2,3,4,5,...|
| -4.438869807456516|(10,[0,1,2,3,4,5,...|
|-19.782762789614537|(10,[0,1,2,3,4,5,...|
| -7.966593841555266|(10,[0,1,2,3,4,5,...|
| -7.896274316726144|(10,[0,1,2,3,4,5,...|
| -8.464803554195287|(10,[0,1,2,3,4,5,...|
| 2.1214592666251364|(10,[0,1,2,3,4,5,...|
| 1.0720117616524107|(10,[0,1,2,3,4,5,...|
|-13.772441561702871|(10,[0,1,2,3,4,5,...|
| -5.082010756207233|(10,[0,1,2,3,4,5,...|
|  7.887786536531237|(10,[0,1,2,3,4,5,...|
| 14.323146365332388|(10,[0,1,2,3,4,5,...|
|-20.057482615789212|(10,[0,1,2,3,4,5,...|
|-0.8995693247765151|(10,[0,1,2,3,4,5,...|
| -19.16829262296376|(10,[0,1,2,3,4,5,...|
|  5.601801561245534|(10,[0,1,2,3,4,5,...|
|-3.2256352187273354|(10,[0,1,2,3,4,5,...|
| 1.5299675726687754|(10,[0,1,2,3,4,5,...|
| -0.250102447941961|(10,[0,1,2,3,4,5,...|
+----------

In [28]:
lr = LinearRegression(featuresCol='features',labelCol='label',predictionCol='prediction',maxIter=320,regParam=10.0)

In [29]:
lrmodel=lr.fit(training)

In [30]:
lrmodel.coefficients

DenseVector([0.0314, 0.359, -0.3916, 1.2052, 0.2368, 0.5966, -0.1743, -0.2588, -0.3266, 0.3267])

In [31]:
lrmodel.intercept

0.2019611307779719

In [32]:
training_summary=lrmodel.summary

In [33]:
training_summary.rootMeanSquaredError

10.200334018050128

In [34]:
training_summary.r2adj

0.0007155071404860713

In [35]:
all_data = spark.read.format('libsvm').load("linearregressiondata/sample_linear_regression_data.txt")

In [36]:
train_data,test_data=all_data.randomSplit([0.7,0.3])

In [37]:
train_data.describe().show()

+-------+-------------------+
|summary|              label|
+-------+-------------------+
|  count|                360|
|   mean|   0.61983691776217|
| stddev| 10.745723939396608|
|    min|-28.571478869743427|
|    max|  27.78383192005107|
+-------+-------------------+



In [38]:
correct_model = lr.fit(train_data)

In [39]:
test_result = correct_model.evaluate(test_data)

In [40]:
test_result.meanAbsoluteError


7.369330535998105

### example1

In [ ]:
data = spark.read.csv('linearregressiondata/Ecommerce_Customers.csv',inferSchema=True,header=True)

In [6]:
data.printSchema()

root
 |-- Email: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- Avatar: string (nullable = true)
 |-- Avg Session Length: double (nullable = true)
 |-- Time on App: double (nullable = true)
 |-- Time on Website: double (nullable = true)
 |-- Length of Membership: double (nullable = true)
 |-- Yearly Amount Spent: double (nullable = true)



In [7]:
data.show(1)

+--------------------+--------------------+------+------------------+-----------------+-----------------+--------------------+-------------------+
|               Email|             Address|Avatar|Avg Session Length|      Time on App|  Time on Website|Length of Membership|Yearly Amount Spent|
+--------------------+--------------------+------+------------------+-----------------+-----------------+--------------------+-------------------+
|mstephenson@ferna...|835 Frank TunnelW...|Violet| 34.49726772511229|12.65565114916675|39.57766801952616|  4.0826206329529615|  587.9510539684005|
+--------------------+--------------------+------+------------------+-----------------+-----------------+--------------------+-------------------+
only showing top 1 row



In [11]:
for item in data.head(1)[0]:
    print(item)

mstephenson@fernandez.com
835 Frank TunnelWrightmouth, MI 82180-9605
Violet
34.49726772511229
12.65565114916675
39.57766801952616
4.0826206329529615
587.9510539684005


In [12]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

In [13]:
data.columns

['Email',
 'Address',
 'Avatar',
 'Avg Session Length',
 'Time on App',
 'Time on Website',
 'Length of Membership',
 'Yearly Amount Spent']

In [14]:
assembler = VectorAssembler(inputCols=['Avg Session Length','Time on App','Time on Website','Length of Membership'],outputCol='features')

In [15]:
output=assembler.transform(data)

In [17]:
output.printSchema()

root
 |-- Email: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- Avatar: string (nullable = true)
 |-- Avg Session Length: double (nullable = true)
 |-- Time on App: double (nullable = true)
 |-- Time on Website: double (nullable = true)
 |-- Length of Membership: double (nullable = true)
 |-- Yearly Amount Spent: double (nullable = true)
 |-- features: vector (nullable = true)



In [19]:
output.head(1)

[Row(Email='mstephenson@fernandez.com', Address='835 Frank TunnelWrightmouth, MI 82180-9605', Avatar='Violet', Avg Session Length=34.49726772511229, Time on App=12.65565114916675, Time on Website=39.57766801952616, Length of Membership=4.0826206329529615, Yearly Amount Spent=587.9510539684005, features=DenseVector([34.4973, 12.6557, 39.5777, 4.0826]))]

In [20]:
final_data=output.select('features','Yearly Amount Spent')

In [21]:
final_data.show()

+--------------------+-------------------+
|            features|Yearly Amount Spent|
+--------------------+-------------------+
|[34.4972677251122...|  587.9510539684005|
|[31.9262720263601...|  392.2049334443264|
|[33.0009147556426...| 487.54750486747207|
|[34.3055566297555...|  581.8523440352177|
|[33.3306725236463...|  599.4060920457634|
|[33.8710378793419...|   637.102447915074|
|[32.0215955013870...|  521.5721747578274|
|[32.7391429383803...|  549.9041461052942|
|[33.9877728956856...|  570.2004089636196|
|[31.9365486184489...|  427.1993848953282|
|[33.9925727749537...|  492.6060127179966|
|[33.8793608248049...|  522.3374046069357|
|[29.5324289670579...|  408.6403510726275|
|[33.1903340437226...|  573.4158673313865|
|[32.3879758531538...|  470.4527333009554|
|[30.7377203726281...|  461.7807421962299|
|[32.1253868972878...| 457.84769594494855|
|[32.3388993230671...| 407.70454754954415|
|[32.1878120459321...|  452.3156754800354|
|[32.6178560628234...|   605.061038804892|
+----------

In [22]:
train_data,test_data=final_data.randomSplit([0.7,0.3])

In [24]:
train_data.describe().show()

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                354|
|   mean|  500.2546041603009|
| stddev|  79.21625940105652|
|    min| 256.67058229005585|
|    max|  744.2218671047146|
+-------+-------------------+



In [25]:
test_data.describe().show()

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                146|
|   mean| 497.03348805855404|
| stddev|  79.77999000289644|
|    min| 302.18954780965197|
|    max|  765.5184619388373|
+-------+-------------------+



In [26]:
lr = LinearRegression(labelCol='Yearly Amount Spent')

In [27]:
lr_model=lr.fit(train_data)

In [28]:
test_results=lr_model.evaluate(test_data)

In [29]:
test_results.residuals.show()

+-------------------+
|          residuals|
+-------------------+
|-12.115328544658837|
| -4.871715085579979|
|  4.264895733189519|
|  -8.54000296715526|
|  21.12742620469089|
|  2.426741833585652|
| -7.648662743533464|
|-18.772102389158817|
| 17.486831447736677|
|-1.2385059389183652|
|-27.130044464122534|
| -2.042455397078527|
|-11.818814449226693|
|-18.959589452559896|
| 7.6212350137498674|
| -5.721447759705541|
| -9.226961030017208|
| -6.794364583289848|
| -4.677804677845529|
|  5.287857544284861|
+-------------------+
only showing top 20 rows



In [30]:
test_results.r2

0.9828480741021159

In [31]:
unlabled_data = test_data.select('features')

In [32]:
unlabled_data.show()

+--------------------+
|            features|
+--------------------+
|[30.3931845423455...|
|[30.8794843441274...|
|[31.0472221394875...|
|[31.1280900496166...|
|[31.2834474760581...|
|[31.3091926408918...|
|[31.4474464941278...|
|[31.5702008293202...|
|[31.6005122003032...|
|[31.6610498227460...|
|[31.6739155032749...|
|[31.7216523605090...|
|[31.8093003166791...|
|[31.8164283341993...|
|[31.8512531286083...|
|[31.8745516945853...|
|[31.8854062999117...|
|[31.9453957483445...|
|[31.9673209478824...|
|[31.9764800614612...|
+--------------------+
only showing top 20 rows



In [33]:
predictions = lr_model.transform(unlabled_data)

In [34]:
predictions.show()

+--------------------+------------------+
|            features|        prediction|
+--------------------+------------------+
|[30.3931845423455...|332.04419834785244|
|[30.8794843441274...|495.07831507043466|
|[31.0472221394875...| 388.2325034558319|
|[31.1280900496166...| 565.7926897142099|
|[31.2834474760581...| 570.6536632209766|
|[31.3091926408918...|  430.293976006348|
|[31.4474464941278...| 426.2514048387575|
|[31.5702008293202...| 564.7175945305637|
|[31.6005122003032...|461.68602004336026|
|[31.6610498227460...| 417.5968595188192|
|[31.6739155032749...|502.85511237400374|
|[31.7216523605090...|349.81938202895117|
|[31.8093003166791...| 548.5907138120679|
|[31.8164283341993...| 520.0820809562163|
|[31.8512531286083...| 465.3710116530485|
|[31.8745516945853...|398.00669200597304|
|[31.8854062999117...| 399.3302340024927|
|[31.9453957483445...| 663.8142885209418|
|[31.9673209478824...| 450.4276459174978|
|[31.9764800614612...|325.30658848981534|
+--------------------+------------